In [ ]:
from pathlib import Path

from syft_rds.orchestra import setup_rds_server

import syft_flwr

In [ ]:
DS = "nutorbitx@gmail.com"
print("DS email: ", DS)

DO1 = "do_1@gmail.com"
DO2 = "do_2@gmail.com"

ds_stack = setup_rds_server(email=DS, key="flwr", root_dir=Path("."))
do_client_1 = ds_stack.init_session(host=DO1)
do_client_2 = ds_stack.init_session(host=DO2)

In [ ]:
do_client_1.is_admin

In [ ]:
do_client_2.is_admin

In [ ]:
do_client_1.datasets

In [ ]:
do_client_2.datasets

In [ ]:
mock_paths = []
for i, client in enumerate([do_client_1, do_client_2]):
    try:
        dataset = client.dataset.get(name=f"marketing-data-{i}")
        mock_paths.append(dataset.get_mock_path())
        print(f"Client {client.host}'s dataset: \n{dataset}\n")
    except Exception as e:
        print(f"Error getting dataset for client {client.host}: {e}")
        mock_paths.append(None)

print(mock_paths)

In [ ]:
mock_paths[0]

In [ ]:
from pathlib import Path

PROJECT_PATH = Path("./vertical-fl")

In [ ]:
PROJECT_PATH

In [ ]:
try:
    (PROJECT_PATH / "main.py").unlink()
    syft_flwr.bootstrap(PROJECT_PATH, aggregator=DS, datasites=[DO1, DO2])
    print("Bootstrapped project successfully ✅")
except Exception as e:
    print(e)

In [ ]:
mock_paths

In [ ]:
syft_flwr.run(PROJECT_PATH, mock_paths)

In [ ]:
for i, client in enumerate([do_client_1, do_client_2]):
    print(f"sending job to {client.host}")
    job = client.jobs.submit(
        name="Syft Flower Experiment",
        description="Syft Flower Federated Learning Experiment",
        user_code_path=PROJECT_PATH,
        dataset_name=f"marketing-data-{i}",
        tags=["federated learning", "fl", "syft_flwr", "flwr"],
        entrypoint="main.py",
    )
    print(job)

# job = do_client_2.jobs.submit(
#     name="Syft Flower Experiment",
#     description="Syft Flower Federated Learning Experiment",
#     user_code_path=PROJECT_PATH,
#     dataset_name=f"fraud-data-2",
#     tags=["federated learning", "fl", "syft_flwr", "flwr"],
#     entrypoint="main.py",
# )
# print(job)

In [ ]:
import os

os.environ["SYFTBOX_CLIENT_CONFIG_PATH"] = str(ds_stack.client.config_path)
os.environ["LOGURU_LEVEL"] = "DEBUG"
os.environ["SYFT_FLWR_MSG_TIMEOUT"] = "60"

!uv run {str(PROJECT_PATH / "main.py")} --active